In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [34]:
batch_size = 128
num_classes = 10
epochs = 10

In [15]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')[:1000]
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train[:1000], num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (1000, 28, 28, 1)
1000 train samples
10000 test samples


In [16]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [17]:
 

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
__________

In [35]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 1000 samples, validate on 10000 samples
Epoch 1/10
1000/1000 [==============================] - 6s 6ms/step - loss: 1.8420 - acc: 0.4300 - val_loss: 1.7352 - val_acc: 0.5577
Epoch 2/10
1000/1000 [==============================] - 6s 6ms/step - loss: 1.8644 - acc: 0.4520 - val_loss: 1.7306 - val_acc: 0.5599
Epoch 3/10
1000/1000 [==============================] - 6s 6ms/step - loss: 1.8323 - acc: 0.4430 - val_loss: 1.7261 - val_acc: 0.5617
Epoch 4/10
1000/1000 [==============================] - 6s 6ms/step - loss: 1.8627 - acc: 0.4250 - val_loss: 1.7216 - val_acc: 0.5644
Epoch 5/10
1000/1000 [==============================] - 5s 5ms/step - loss: 1.8371 - acc: 0.4510 - val_loss: 1.7173 - val_acc: 0.5660
Epoch 6/10
1000/1000 [==============================] - 5s 5ms/step - loss: 1.8355 - acc: 0.4440 - val_loss: 1.7129 - val_acc: 0.5678
Epoch 7/10
1000/1000 [==============================] - 6s 6ms/step - loss: 1.8134 - acc: 0.4650 - val_loss: 1.7085 - val_acc: 0.5696
Epoch 8/10
10

In [19]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.758016736984253
Test accuracy: 0.5484


In [20]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [29]:
'''
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
'''

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adadelta(lr=lr_schedule(0)),
              metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import numpy as np

lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)


Learning rate:  0.001


In [36]:
callbacks = [lr_scheduler, lr_reducer]

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=callbacks)

Train on 1000 samples, validate on 10000 samples
Epoch 1/10
Learning rate:  0.001
1000/1000 [==============================] - 6s 6ms/step - loss: 1.7801 - acc: 0.4550 - val_loss: 1.6913 - val_acc: 0.5775
Epoch 2/10
Learning rate:  0.001
1000/1000 [==============================] - 6s 6ms/step - loss: 1.8111 - acc: 0.4510 - val_loss: 1.6869 - val_acc: 0.5792
Epoch 3/10
Learning rate:  0.001
1000/1000 [==============================] - 6s 6ms/step - loss: 1.8048 - acc: 0.4480 - val_loss: 1.6826 - val_acc: 0.5807
Epoch 4/10
Learning rate:  0.001
1000/1000 [==============================] - 6s 6ms/step - loss: 1.7761 - acc: 0.4590 - val_loss: 1.6784 - val_acc: 0.5824
Epoch 5/10
Learning rate:  0.001
1000/1000 [==============================] - 5s 5ms/step - loss: 1.7672 - acc: 0.4630 - val_loss: 1.6742 - val_acc: 0.5854
Epoch 6/10
Learning rate:  0.001
1000/1000 [==============================] - 6s 6ms/step - loss: 1.7701 - acc: 0.4730 - val_loss: 1.6700 - val_acc: 0.5876
Epoch 7/10
Lear

In [31]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.748862872314453
Test accuracy: 0.552
